### Split KITTI dataset annotations files to match JDE model format

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import time

In [2]:
def readFileByLines(filepath):
    with open(filepath) as f:
        content = f.readlines()
    return [x.strip() for x in content]

In [3]:
def convertBbox(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    y = y*dh
    w = w*dw
    h = h*dh
    return (x,y,w,h)

In [4]:
def createDFRow(identity, bbox):
    return {'class': '0',
            'identity': str(identity),
            'x_center': str(format(bbox[0],'f')),
            'y_center': str(format(bbox[1],'f')),
            'width': str(format(bbox[2],'f')),
            'height': str(format(bbox[3],'f'))}

In [5]:
anno_folder = './KITTI_ANNO'
img_folder = './KITTI_IMG'
out_folder = './KITTI_OUT_ANNO'
anno_files = os.listdir(anno_folder)
img_dir = os.listdir(img_folder)
anno_dir = os.listdir(out_folder)

In [6]:
anno_out_dir = []
for dir in anno_dir:
    anno_out_dir.append(os.path.join(out_folder, dir))
anno_out_dir

['./KITTI_OUT_ANNO/0000',
 './KITTI_OUT_ANNO/0001',
 './KITTI_OUT_ANNO/0002',
 './KITTI_OUT_ANNO/0003',
 './KITTI_OUT_ANNO/0004',
 './KITTI_OUT_ANNO/0005',
 './KITTI_OUT_ANNO/0006',
 './KITTI_OUT_ANNO/0007',
 './KITTI_OUT_ANNO/0008',
 './KITTI_OUT_ANNO/0009',
 './KITTI_OUT_ANNO/0010',
 './KITTI_OUT_ANNO/0011',
 './KITTI_OUT_ANNO/0012',
 './KITTI_OUT_ANNO/0013',
 './KITTI_OUT_ANNO/0014',
 './KITTI_OUT_ANNO/0015',
 './KITTI_OUT_ANNO/0016',
 './KITTI_OUT_ANNO/0018',
 './KITTI_OUT_ANNO/0019',
 './KITTI_OUT_ANNO/0020']

In [7]:
anno_file_path = []
for anno in anno_files:
    anno_file_path.append(os.path.join(anno_folder, anno))

In [8]:
anno_file_path

['./KITTI_ANNO/0000.txt',
 './KITTI_ANNO/0001.txt',
 './KITTI_ANNO/0002.txt',
 './KITTI_ANNO/0003.txt',
 './KITTI_ANNO/0004.txt',
 './KITTI_ANNO/0005.txt',
 './KITTI_ANNO/0006.txt',
 './KITTI_ANNO/0007.txt',
 './KITTI_ANNO/0008.txt',
 './KITTI_ANNO/0009.txt',
 './KITTI_ANNO/0010.txt',
 './KITTI_ANNO/0011.txt',
 './KITTI_ANNO/0012.txt',
 './KITTI_ANNO/0013.txt',
 './KITTI_ANNO/0014.txt',
 './KITTI_ANNO/0015.txt',
 './KITTI_ANNO/0016.txt',
 './KITTI_ANNO/0018.txt',
 './KITTI_ANNO/0019.txt',
 './KITTI_ANNO/0020.txt']

In [9]:
img_size = []
for dir in img_dir:
    fullpath = os.path.join(img_folder, os.path.join(dir, '000000.png'))
    img = cv2.imread(fullpath)
    img_size.append([img.shape[1], img.shape[0]])

In [10]:
img_size

[[1242, 375],
 [1242, 375],
 [1242, 375],
 [1242, 375],
 [1242, 375],
 [1242, 375],
 [1242, 375],
 [1242, 375],
 [1242, 375],
 [1242, 375],
 [1242, 375],
 [1242, 375],
 [1242, 375],
 [1242, 375],
 [1224, 370],
 [1224, 370],
 [1224, 370],
 [1238, 374],
 [1238, 374],
 [1241, 376]]

In [11]:
df_list = []
col = ['frameId', 'trackId', 'objectType', 'truncated', 'occluded', 'alpha', 'bbox_left', 'bbox_top', 'bbox_right', 'bbox_bottom', 'height', 'width', 'length', 'x', 'y', 'z', 'ry']
for path in anno_file_path:
    df_list.append(pd.read_csv(path, sep=' ', names=col))

In [12]:
df_list[0]

,frameId,trackId,objectType,truncated,occluded,alpha,bbox_left,bbox_top,bbox_right,bbox_bottom,height,width,length,x,y,z,ry
0,0,-1,DontCare,-1,-1,-10.000000,219.310000,188.490000,245.500000,218.560000,-1000.000000,-1000.000000,-1000.000000,-10.000000,-1.000000,-1.000000,-1.000000
1,0,-1,DontCare,-1,-1,-10.000000,47.560000,195.280000,115.480000,221.480000,-1000.000000,-1000.000000,-1000.000000,-10.000000,-1.000000,-1.000000,-1.000000
2,0,0,Van,0,0,-1.793451,296.744956,161.752147,455.226042,292.372804,2.000000,1.823255,4.433886,-4.552284,1.858523,13.410495,-2.115488
3,0,1,Cyclist,0,0,-1.936993,737.619499,161.531951,931.112229,374.000000,1.739063,0.824591,1.785241,1.640400,1.675660,5.776261,-1.675458
4,0,2,Pedestrian,0,0,-2.523309,1106.137292,166.576807,1204.470628,323.876144,1.714062,0.767881,0.972283,6.301919,1.652419,8.455685,-1.900245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1084,153,10,Car,0,2,-1.818856,680.294919,177.511028,842.313244,284.070033,1.524000,1.728591,3.894227,2.353367,1.622590,12.436503,-1.637280
1085,153,11,Car,0,2,1.864481,245.920800,194.456182,394.817829,286.444967,1.444000,1.595116,3.791789,-5.458963,1.908188,13.979427,1.497916
1086,153,12,Pedestrian,1,0,0.826456,1185.199080,151.165841,1241.000000,348.552707,1.688000,0.800000,0.884000,5.739732,1.500532,6.279632,1.543272
1087,153,13,Car,0,0,1.773993,344.361560,188.772369,430.531955,248.482384,1.422414,1.512803,3.707634,-6.033258,1.888008,19.788795,1.481180


In [13]:
%%time
out_cols = ['class', 'identity', 'x_center', 'y_center', 'width', 'height']
veh_filter = ['Car', 'Van', 'Truck']
df_list_mod = []

for i in range(len(df_list)):
    df = df_list[i]
    
    # Set last frameId
    lastFrameId = df['frameId'].iloc[-1]
    print(lastFrameId)
    # Drop unused column
    df.drop(['truncated', 'occluded', 'alpha', 'height', 'width', 'length', 'x', 'y', 'z', 'ry'], axis=1, inplace=True)

    # Filter only vehicles
    df = df[df['objectType'].isin(veh_filter)]

    # Set folder path
    folder = anno_out_dir[i]
    
    # Set image size
    size = img_size[i]

    # Loop rows in dataframe
    for rows in range(lastFrameId+1):
        out_df = pd.DataFrame(columns=out_cols)
        file_name = os.path.join(folder, str(rows).zfill(6) + '.txt')
        frows = df.loc[df['frameId'] == rows]
        if frows.shape[0] == 0:
            np.savetxt(file_name, out_df, fmt='%s')
            continue
        else:
            for _, subrows in frows.iterrows():
                bbox = convertBbox(size, [subrows.bbox_left, subrows.bbox_right, subrows.bbox_top, subrows.bbox_bottom])
                newrow = createDFRow(subrows.trackId, bbox)
                out_df = out_df.append(newrow, ignore_index=True)
            np.savetxt(file_name, out_df, fmt='%s')


    # df_list_mod.append(df)


153
446
232
143
313
296
269
799
389
802
293
372
77
339
105
375
208
338
1058
836
CPU times: user 2min 32s, sys: 4.94 s, total: 2min 37s
Wall time: 3min 10s
